In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
sns.set_style('darkgrid')

In [ ]:
import sys
sys.path.append('../../pytpc')
import pytpc

# Simulated data

In [ ]:
pt = pytpc.Particle(4., 2., 3.)

In [ ]:
gas = pytpc.gases.HeliumGas(150.-15.)

In [ ]:
dedx = []
for e in np.logspace(-3, 3, 50):
    pt.energy = e
    dedx.append([e, gas.energy_loss(e, pt.mass_num, pt.charge_num) / 100 / gas.density])
simdat = pd.DataFrame(dedx, columns=('energy', 'dedx'))

In [ ]:
plt.plot(simdat.energy, simdat.dedx)
plt.ylabel('dE/dx [MeV/(g/cm^2)]')
plt.xlabel('Particle Energy [MeV]')

In [ ]:
gas.energy_loss(10, 4, 2)

In [ ]:
gas.density

In [ ]:
oldgas = pytpc.Gas(4, 2, 41.8, 150)
oldgas.energy_loss(10, 4, 2)

# ASTAR data

In [ ]:
import re
import pandas as pd

In [ ]:
with open('/Users/josh/Documents/Data/GasData/helium.txt') as f:
    hedata = []
    for line in f:
        elmts = line.strip().split()
        if any([re.match(r'\d*\.\d*E[+-]\d*', x) for x in elmts]):
            hedata.append([float(x) for x in elmts])
    hedata = pd.DataFrame(hedata, columns=('energy', 'dedx'))

In [ ]:
with open('/Users/josh/Documents/Data/GasData/co2.txt') as f:
    codata = []
    for line in f:
        elmts = line.strip().split()
        if any([re.match(r'\d*\.\d*E[+-]\d*', x) for x in elmts]):
            codata.append([float(x) for x in elmts])
    codata = np.array(codata)
    codata = pd.DataFrame(codata, columns=('energy', 'dedx'))

In [ ]:
plt.plot(hedata.energy, hedata.dedx)
plt.plot(codata.energy, codata.dedx)
plt.plot(simdat.energy, simdat.dedx)
plt.loglog();

In [ ]:
def hepoly(e):
    a0 = -5.083e-2
    a1 = 2.502e-1
    a2 = 6.796e-2
    a3 = 1.620e-3
    return a0 + a1 * e + a2 * e**2 + a3 * e**3

def hestop(e):
    return 1000 / (0.2502 + 0.13592*e + 0.00486*e**2)

def co2poly(e):
    a0 = 4.848e-3
    a1 = 4.516e-1
    a2 = 7.192e-3
    a3 = 2.587e-2
    a4 = -2.400e-3
    return a0 + a1 * e + a2 * e**2 + a3 * e**3 + a4 * e**4

def co2stop(e):
    return 1000 / (0.4516 + 0.014384*e + 0.07761*e**2 - 0.0096*e**3)

In [ ]:
xs = np.linspace(0.5, 5.3, 50)
plt.plot(xs, hestop(xs), label='Brendle')
plt.plot(hedata.energy, hedata.dedx, label='ASTAR')
plt.plot(simdat.energy, simdat.dedx, label='Wolfi')
plt.xlim(0.1, 100)
plt.loglog()
plt.legend()

In [ ]:
plt.plot(xs, co2stop(xs), label='Brendle')
plt.plot(codata.energy, codata.dedx, label='ASTAR')
plt.legend()
plt.loglog()

In [ ]:
co2stop(1000)